<a href="https://colab.research.google.com/github/talitmr/text_preprocess_study/blob/main/lemmatization_and_postprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lemmatization and Post Processing

In this part, I tried to improve the preprocessing parts using stemming, lemmatization and both. 

In [ ]:
import torch
print(torch.__version__) #version of the pytorh
import torch.nn.functional as F
import torchtext.legacy as torchtext
import random


from nltk.stem import SnowballStemmer

import pandas as pd
import html
from nltk import download

from nltk.tokenize import TweetTokenizer

from nltk import pos_tag
from nltk.data import load
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

from nltk.corpus import stopwords
import sys
import re
import nltk.data
from nltk import pos_tag_sents
# If you have Anaconda, you can install emoji using 
# "conda install -c conda-forge emoji" command. You can download autocorrect using pip
# and "target" parameter: "pip install autocorrect --target=<directory>"

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from snowballstemmer import TurkishStemmer
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

1.8.1+cu101
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

## Finding Max Length 


First thing we need to do is finding the max length of the text in the dataset because our input size is constant.

In [ ]:
## In this cell create "tokenize_fn" that takes text and returns tokens
##Example
## def tokenize_fn(text):
##   tokens = do_tokenize(text)
##   return tokens
from nltk.tokenize import RegexpTokenizer

def tokenize_fn(text):

  tokenizer = RegexpTokenizer(r'\w+')

  return tokenizer.tokenize(text)

In [ ]:
def stemming_fn(text):
  stemmer = SnowballStemmer("english")
  sentence_stems = [stemmer.stem(token) for token in text]

  return sentence_stems

In [ ]:

lemmatizer = WordNetLemmatizer()
 
def get_lemmatizer_pos(pos):
  pos_start = pos[0]# Takes the first letter to simplify the POS tag
  if pos_start == "J":
    return wn.ADJ
  elif pos_start == "V":
   return wn.VERB
  elif pos_start == "R":
   return wn.ADV
  else:
    return wn.NOUN 
 
 
def lemmatization_fn(text):
  sentence_tokens_pos = pos_tag(text)
  sentence_lemmas = [lemmatizer.lemmatize(token[0], pos=get_lemmatizer_pos(token[1]))for token in sentence_tokens_pos]
  return sentence_lemmas



In [ ]:
TEXT = torchtext.data.Field(tokenize=tokenize_fn, postprocessing = lemmatization_fn, batch_first=True, lower = True) # preprocessing paraneters can be used to add aditional  preprocessing steps
LABEL = torchtext.data.LabelField(dtype = torch.float)

In [ ]:
train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:04<00:00, 18.8MB/s]


In [ ]:
print(vars(train_data[0]))

{'text': ['i', 'f', 'you', 'thought', 'sam', 'mendes', 'first', 'film', 'the', 'much', 'heralded', 'american', 'beauty', 'was', 'a', 'movie', 'with', 'style', 'to', 'spare', 'wait', 'until', 'you', 'see', 'his', 'highly', 'anticipated', 'second', 'effort', 'the', 'unrelentingly', 'grim', '30', 's', 'gangster', 'melodrama', 'road', 'to', 'perdition', 'some', 'critics', 'have', 'hailed', 'this', 'new', 'movie', 'as', 'a', 'worthy', 'successor', 'to', 'the', 'godfather', 'a', 'rash', 'judgment', 'made', 'by', 'several', 'reviewers', 'taken', 'with', 'mr', 'mendes', 'extraordinary', 'technical', 'prowess', 'if', 'the', 'mechanics', 'of', 'movie', 'making', 'are', 'what', 'make', 'a', 'picture', 'great', 'then', 'yes', 'road', 'to', 'perdition', 'is', 'a', 'distant', 'cousin', 'to', 'the', 'godfather', 'in', 'terms', 'of', 'what', 'it', 'achieves', 'in', 'cinematography', 'editing', 'music', 'scoring', 'and', 'sound', 'what', 'it', 'doesn', 't', 'have', 'is', 'a', 'resonance', 'that', 'all'

In [ ]:
max_size=0  ## this part of the code find maximum length of the network
count=0
sum= 0
for i in  range(len(train_data)):
  if max_size < len(train_data[i].text):
    max_size =len(train_data[i].text)
    print(max_size)
  count +=1
  sum +=len(train_data[0].text)
print("avarage: ", sum/count)

559
623
1051
1055
1107
1428
1666
1776
1866
1873
2525
avarage:  559.0


## Training Embeddings

We just have found the max length corpus. Now let's create our train and test datasets.

In [ ]:
TEXT = torchtext.data.Field(tokenize=tokenize_fn, preprocessing= lemmatization_fn, batch_first=True,fix_length=max_size, lower=True ) # preprocessing parameters can be used to add aditional  preprocessing steps
LABEL = torchtext.data.LabelField(dtype = torch.float)
train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL) 
print("train length is: ",len(train_data))
print("test length is: ",len(test_data))
print(vars(train_data[0]))

train length is:  25000
test length is:  25000
{'text': ['i', 'f', 'you', 'think', 'sam', 'mendes', 'first', 'film', 'the', 'much', 'herald', 'american', 'beauty', 'be', 'a', 'movie', 'with', 'style', 'to', 'spare', 'wait', 'until', 'you', 'see', 'his', 'highly', 'anticipated', 'second', 'effort', 'the', 'unrelentingly', 'grim', '30', 's', 'gangster', 'melodrama', 'road', 'to', 'perdition', 'some', 'critic', 'have', 'hail', 'this', 'new', 'movie', 'a', 'a', 'worthy', 'successor', 'to', 'the', 'godfather', 'a', 'rash', 'judgment', 'make', 'by', 'several', 'reviewer', 'take', 'with', 'mr', 'mendes', 'extraordinary', 'technical', 'prowess', 'if', 'the', 'mechanic', 'of', 'movie', 'making', 'be', 'what', 'make', 'a', 'picture', 'great', 'then', 'yes', 'road', 'to', 'perdition', 'be', 'a', 'distant', 'cousin', 'to', 'the', 'godfather', 'in', 'term', 'of', 'what', 'it', 'achieve', 'in', 'cinematography', 'edit', 'music', 'scoring', 'and', 'sound', 'what', 'it', 'doesn', 't', 'have', 'be', 'a

In [ ]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [ ]:
print("Unique tokens in TEXT vocabulary:",len(TEXT.vocab))
print("Unique tokens in LABEL vocabulary:",len(LABEL.vocab))
print(TEXT.vocab.freqs.most_common(20))
print(LABEL.vocab.freqs)
print(TEXT.unk_token)
print(TEXT.pad_token)

Unique tokens in TEXT vocabulary: 64737
Unique tokens in LABEL vocabulary: 2
[('the', 336749), ('be', 241407), ('a', 203889), ('and', 164140), ('of', 145864), ('to', 135724), ('it', 104648), ('br', 101871), ('in', 94015), ('i', 87690), ('this', 76007), ('that', 73286), ('s', 63633), ('have', 58357), ('movie', 51715), ('film', 47940), ('for', 44345), ('with', 44130), ('but', 42623), ('t', 34390)]
Counter({'pos': 12500, 'neg': 12500})
<unk>
<pad>


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##Create a train and test iterators using Bucket iterator method with batch size 32
train_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, test_data), batch_size=32, device=device)

Let's create our network;

In [ ]:
class Network(torch.nn.Module):
    def __init__(self,pad_idx):
        super().__init__()
        self.embedding = torch.nn.Embedding(num_embeddings = len(TEXT.vocab), embedding_dim = 100,padding_idx = pad_idx)
        self.layer1 = torch.nn.Linear(max_size*100, 1)

    def forward(self, x):
        x = self.embedding(x).view(x.size(0),-1)
        x = self.layer1(x)
        return x       

In [ ]:
model = Network(pad_idx = TEXT.vocab.stoi[TEXT.pad_token])
print(model)

Network(
  (embedding): Embedding(64737, 100, padding_idx=1)
  (layer1): Linear(in_features=252500, out_features=1, bias=True)
)


In [ ]:
# Choose a Loss function from torch.nn according to your network
loss_fn = torch.nn.BCEWithLogitsLoss() 

#Define an Adam optimizer with learning rate 0.001 to optimize the parameters of our network
optimizer = torch.optim.Adam(params= model.parameters(),lr= 0.001)

In [ ]:
model = model.to(device)
loss_fn = loss_fn.to(device)

We can train our network now.

In [ ]:
def accuracy_fn(predictions, labels):  ## create a accuraccy function for further use
  correct = (torch.round(torch.sigmoid(predictions)) == labels).float() 
  accuracy = correct.sum() / len(correct)
  return accuracy

In [ ]:
import time
# Training loop
N_EPOCHS = 2

tr_loss = []
model.train()

for epoch in range(N_EPOCHS):
    
    # Calculate training time
    start_time = time.time()

    epoch_loss = 0
    epoch_acc = 0

    
    batch_no = 0
    for batch in train_iterator:
        predictions = model(batch.text).squeeze(1)
        loss = loss_fn(predictions, batch.label.squeeze(0))

        optimizer.zero_grad()# Reset the gradients
        loss.backward()# Backprop   
        optimizer.step()# Optimize the weights
        ##################################

        # Record accuracy and loss
        epoch_loss += loss.item()

        acc =   accuracy_fn(predictions,batch.label.squeeze(0)) 
        epoch_acc +=acc.item()

        batch_no = batch_no +1
        
        if batch_no%60 == 0:
          print(f'Epoch:  {epoch+1:2} | Batch No: {batch_no} | Loss: {loss.item():.3f} | Accuracy: {acc.item()*100:.2f}%')

    
    train_loss = epoch_loss / len(train_iterator)

    end_time = time.time()

    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    print('\n')    
    print(f'Epoch: {epoch+1:2} | Epoch Time: {elapsed_mins}m {elapsed_secs}s')
    print(f'\tAvarage Train Loss: {train_loss:.3f} ')
    print('\n') 

Epoch:   1 | Batch No: 60 | Loss: 0.862 | Accuracy: 59.38%
Epoch:   1 | Batch No: 120 | Loss: 1.122 | Accuracy: 56.25%
Epoch:   1 | Batch No: 180 | Loss: 1.010 | Accuracy: 59.38%
Epoch:   1 | Batch No: 240 | Loss: 0.874 | Accuracy: 62.50%
Epoch:   1 | Batch No: 300 | Loss: 1.448 | Accuracy: 40.62%
Epoch:   1 | Batch No: 360 | Loss: 1.394 | Accuracy: 53.12%
Epoch:   1 | Batch No: 420 | Loss: 1.091 | Accuracy: 62.50%
Epoch:   1 | Batch No: 480 | Loss: 1.095 | Accuracy: 50.00%
Epoch:   1 | Batch No: 540 | Loss: 0.807 | Accuracy: 65.62%
Epoch:   1 | Batch No: 600 | Loss: 0.740 | Accuracy: 71.88%
Epoch:   1 | Batch No: 660 | Loss: 0.609 | Accuracy: 68.75%
Epoch:   1 | Batch No: 720 | Loss: 1.684 | Accuracy: 50.00%
Epoch:   1 | Batch No: 780 | Loss: 0.728 | Accuracy: 75.00%


Epoch:  1 | Epoch Time: 1m 30s
	Avarage Train Loss: 1.008 


Epoch:   2 | Batch No: 60 | Loss: 0.317 | Accuracy: 90.62%
Epoch:   2 | Batch No: 120 | Loss: 0.842 | Accuracy: 81.25%
Epoch:   2 | Batch No: 180 | Loss: 0.32

In [ ]:
test_epoch_loss = 0
test_epoch_acc = 0

# Turm on evalutaion mode
model.eval()

# No need to backprop in eval
with torch.no_grad():

    for batch in test_iterator:

        test_predictions = model(batch.text).squeeze(1)
        
        test_loss = loss_fn(test_predictions, batch.label)

        test_epoch_loss += test_loss.item()
        
        acc = accuracy_fn(test_predictions,batch.label.squeeze(0))
        test_epoch_acc +=acc.item()

test_loss = test_epoch_loss/len(test_iterator)
test_acc = test_epoch_acc  / len(test_iterator)
print(f'Test Loss: {test_loss:.3f} | | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.031 | | Test Acc: 70.09%
